<a href="https://colab.research.google.com/github/CatLads/Notebooks/blob/main/DDDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install flatland-rl tqdm

In [ ]:
def normalize_observation(observation):
    return observation.flatten()

In [ ]:
from scipy import signal, ndimage
from flatland.envs.observations import ObservationBuilder
import numpy as np

class TemperatureObservation(ObservationBuilder):
    #self.rail_obs = np.zeros((self.env.height, self.env.width, len(self.env.agents)))

    def reset(self):
        self.rail_obs = np.zeros((self.env.height, self.env.width, len(self.env.agents)))
        return self.rail_obs, {}
               
    def relax_temperature(self, handle: int = 0):
        """
        Relax the temperature
        """
        relaxed = ndimage.gaussian_filter(self.rail_obs[:, :, handle], 1)
        print(self.env)
        mask = self.env.rail.grid > 0
        relaxed[~mask] = np.inf
        return relaxed

    def get(self, handle: int = 0) -> (np.ndarray):
        agent = self.env.agents[handle]
        x_target = agent.target[0]
        y_target = agent.target[1]
        for i, other_agent in enumerate(self.env.agents): # This sets the temperatures for other 🚂s (which are hot)
            x = other_agent.position[0] if other_agent.position is not None else None
            y = other_agent.position[1] if other_agent.position is not None else None
            if handle != i:
                self.rail_obs[x, y, i] = 1
            
        self.rail_obs[x_target, y_target, handle] = -1

        self.relax_temperature(handle)
        return self.rail_obs[:, :, handle]

In [ ]:
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.schedule_generators import sparse_schedule_generator
from flatland.envs.rail_env import RailEnv, RailEnvActions
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.observations import GlobalObsForRailEnv, TreeObsForRailEnv
from gym import spaces
from tqdm import tqdm
import os
import sys # I'm stuck


seed = 69 #nice 


width = 10 # @param{type: "integer"}
height = 10 # @param{type: "integer"}
num_agents =  3  # @param{type: "integer"}
tree_depth = 2 # @param{type: "integer"}
radius_observation = 10
WINDOW_LENGTH =   22# @param{type: "integer"}


random_rail_generator = complex_rail_generator(
    nr_start_goal=10, # @param{type:"integer"} number of start and end goals 
                      # connections, the higher the easier it should be for 
                      # the trains
    nr_extra=10, # @param{type:"integer"} extra connections 
                 # (useful for alternite paths), the higher the easier
    min_dist=10,
    max_dist=99999,
    seed=seed
)

env = RailEnv(
    width=width,
    height=height,
    rail_generator=random_rail_generator,
    obs_builder_object=TemperatureObservation(),
    number_of_agents=num_agents
)

obs, info = env.reset()


state_shape = normalize_observation(obs[0]).shape
action_shape = (5,)
#normalize_observation(obs[0], tree_depth, radius_observation)

In [ ]:
import tensorflow as tf 
import numpy as np 
from tensorflow.keras.models import load_model

In [ ]:
class DDDQN(tf.keras.Model):
    def __init__(self):
      super(DDDQN, self).__init__()
      self.d1 = tf.keras.layers.Dense(128, activation='relu')
      self.d2 = tf.keras.layers.Dense(128, activation='relu')
      self.v = tf.keras.layers.Dense(1, activation=None)
      self.a = tf.keras.layers.Dense(5, activation=None) # TODO: Change this please

    def call(self, input_data):
      x = self.d1(input_data)
      x = self.d2(x)
      v = self.v(x)
      a = self.a(x)
      Q = v +(a -tf.math.reduce_mean(a, axis=1, keepdims=True))
      return Q

    def advantage(self, state):
      x = self.d1(state)
      x = self.d2(x)
      a = self.a(x)
      return a


In [ ]:
class exp_replay():
    def __init__(self, buffer_size= 1000000):
        self.buffer_size = buffer_size

        self.state_mem = np.zeros((self.buffer_size, *state_shape), dtype=np.float32)
        self.action_mem = np.zeros((self.buffer_size), dtype=np.int32) #SIMMY: I removed , *action_shape cause we should just save one action per agent, not all of them
        self.reward_mem = np.zeros((self.buffer_size), dtype=np.float32)
        self.next_state_mem = np.zeros((self.buffer_size, *state_shape), dtype=np.float32)
        self.done_mem = np.zeros((self.buffer_size), dtype=np.bool)
        self.pointer = 0

    def add_exp(self, state, action, reward, next_state, done):
        idx  = self.pointer % self.buffer_size 
        self.state_mem[idx] = state
        self.action_mem[idx] = action
        self.reward_mem[idx] = reward
        self.next_state_mem[idx] = next_state
        self.done_mem[idx] = 1 - int(done)
        self.pointer += 1

    def sample_exp(self, batch_size= 64):
        max_mem = min(self.pointer, self.buffer_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)
        states = self.state_mem[batch]
        actions = self.action_mem[batch]
        rewards = self.reward_mem[batch]
        next_states = self.next_state_mem[batch]
        dones = self.done_mem[batch]
        return states, actions, rewards, next_states, dones


In [ ]:
class agent():
      def __init__(self, gamma=0.99, replace=100, lr=0.001):
          self.gamma = gamma
          self.epsilon = 1.0
          self.min_epsilon = 0.01
          self.epsilon_decay = 1e-3
          self.replace = replace
          self.trainstep = 0
          self.memory = exp_replay()
          self.batch_size = 64
          self.q_net = DDDQN()
          self.target_net = DDDQN()
          opt = tf.keras.optimizers.Adam(learning_rate=lr)
          self.q_net.compile(loss='mse', optimizer=opt)
          self.target_net.compile(loss='mse', optimizer=opt)


      def act(self, state):
          if np.random.rand() <= self.epsilon:
              return np.random.choice([i for i in range(5)]) # TODO: change the 5

          else:
              actions = self.q_net.advantage(np.array([state]))
              action = np.argmax(actions)
              return action

      def update_mem(self, state, action, reward, next_state, done):
          self.memory.add_exp(state, action, reward, next_state, done)

      def update_target(self):
          self.target_net.set_weights(self.q_net.get_weights())     

      def update_epsilon(self):
          self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.min_epsilon else self.min_epsilon
          return self.epsilon
   
      def train(self):
          if self.memory.pointer < self.batch_size:
             return 
          
          if self.trainstep % self.replace == 0:
             self.update_target()
          states, actions, rewards, next_states, dones = self.memory.sample_exp(self.batch_size)
          target = self.q_net.predict(states)
          next_state_val = self.target_net.predict(next_states)
          print(next_states.shape, target.shape)
          max_action = np.argmax(self.q_net.predict(next_states), axis=1)
          batch_index = np.arange(self.batch_size, dtype=np.int32)
          q_target = np.copy(target)
          print((q_target[batch_index, actions]).shape)
          q_target[batch_index, actions] = rewards + self.gamma * next_state_val[batch_index, max_action]*dones
          self.q_net.train_on_batch(states, q_target)
          self.update_epsilon()
          self.trainstep += 1

      def save_model(self):
          self.q_net.save_weights("model")
          self.target_net.save_weights("target_model")
          print("model saved")

      def load_model(self):
          self.q_net.load_weights("model")
          self.target_net.load_weights("model")
          print("model loaded")

In [ ]:
agent007 = agent()
#if(glob.glob("model*") != []):
#    agent007.load_model()


In [ ]:


saving_interval = 50
# Train for 300 episodes
for episode in range(3000):
    try:
        # Initialize episode
        states, info = env.reset()
        #env_renderer = RenderTool(env)
        done = {i: False for i in range(0, num_agents)}
        done["__all__"] = False
        scores = 0
        scores_window = []

        
        while not done["__all__"]:
            actions = {}
            agents_obs = {}
            
            for i in range(0, num_agents):
                if not done[i]:
                    agents_obs[i] = normalize_observation(states[i])
                    actions[i] = agent007.act(agents_obs[i])
                
                    
            next_obs, all_rewards, done, info = env.step(actions) #base env
            #env_renderer.render_env(show=True)
            for i in range(num_agents):
                if not done[i]:
                    normalized_next_obs = normalize_observation(next_obs[i])
                    agent007.update_mem(agents_obs[i], actions[i], all_rewards[i], normalized_next_obs, done[i])
                elif i in agents_obs: #SIMMY: It was just an else. It worked on the first loop after the "done", but then we set agents_obs to {} so it wasn't filled anymore!
                    agent007.update_mem(agents_obs[i], actions[i], all_rewards[i], agents_obs[i], done[i])
                scores += all_rewards[i]/num_agents
            agent007.train()
            scores_window.append(scores/-75.5)
        print(f"Final reward in episode {episode} was {np.mean(scores_window)}")
        if(episode%saving_interval == 0):
            agent007.save_model()
    except KeyboardInterrupt:
        print('Interrupted')
        agent007.save_model()
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)
    #sum_rewards += reward


(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
Final reward in episode 1 was 0.9999999999999978
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 5)
(64,)
(64, 100) (64, 

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8b2a743-4403-48d9-b1f8-a1215902878c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>